In [1]:
import os
os.environ['AUTOGRAPH_VERBOSITY'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import tensorflow as tf
tf.enable_eager_execution()
tf.get_logger().setLevel('ERROR')

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from nasbench.lib import cifar
from nasbench.lib import model_builder
from nasbench.lib import training_time
import numpy as np


from nasbench.lib import evaluate
from nasbench.lib import model_spec
from nasbench.lib import config as _config

from nasbench.scripts.run_evaluation import NumpyEncoder

import json
import functools

In [4]:
config = _config.build_config()
config['use_tpu'] = False
config['train_data_files'] = ['../data/dataset/train_1.tfrecords', '../data/dataset/train_2.tfrecords', '../data/dataset/train_3.tfrecords', '../data/dataset/train_4.tfrecords']
config['valid_data_file'] = ['../data/dataset/validation.tfrecords']
config['test_data_file'] = ['../data/dataset/test.tfrecords']
config['sample_data_file'] = ['../data/dataset/sample.tfrecords']
config['train_epochs'] = 36

matrix=[[0, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]] 
# Operations at the vertices of the module, matches order of matrix
labels=['input', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'output']


matrix = np.array(matrix)
labels = np.array(labels)

spec = model_spec.ModelSpec(matrix, labels)
model_dir = '../data/donor_data'

In [5]:
run_config = tf.contrib.tpu.RunConfig(model_dir=model_dir)

# check model accuracy

In [6]:
input_test = cifar.CIFARInput('test', config)

In [7]:
est = tf.contrib.tpu.TPUEstimator(
      use_tpu=config['use_tpu'],
      model_fn=model_builder.build_model_fn(spec, config, None),
      config=run_config,
      params={'use_KD':False},
      train_batch_size=config['batch_size'],
      eval_batch_size=config['batch_size'],
      predict_batch_size=1000)

In [8]:
est_preds = est.predict(input_fn=input_test.input_fn, yield_single_examples=False)

In [9]:
all_pred_logits = []
for preds in est_preds:
    all_pred_logits.append(preds['logits'])
all_pred_logits = np.vstack(all_pred_logits)

Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8b64268>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Whitelisted: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8b64268>: DoNotConvert rule for tensorflow
Converted call: functools.partial(<function _parser at 0x7f68c4d09158>, False)
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Not whitelisted: <method-wrapper '__call__' of functools.partial object at 0x7f68b8bd19f8>: default rule
Not whitelisted: functools.partial(<function _parser at 0x7f68c4d09158>, False): default rule
Entity <function _parser at 0x7f68c4d09158> is not cached for key <code object _parser at 0x7f68c4cecae0, file "/usr/local/lib/python3.6/dist-packages/nasbench-1.0-py3.6.egg/nasbench/lib/cifar.py", line 250> subkey (<tensorflow.python.autograph.core.converter.ConversionOptions object at 0x7f68b8bdc4e0>, frozenset())
Converting <function _parser at 0x7f68c4d09

In [ ]:
preds = np.argmax(all_pred_logits, axis=1)

In [ ]:
preds

In [ ]:
dataset = input_test.input_fn({'batch_size':1000, 'use_KD':False})

In [ ]:
answers = []
cnt = 0
for records in dataset:
    answers.append(records[1].numpy())
    cnt += 1
answers = np.hstack(answers)

In [ ]:
cnt

In [ ]:
answers

In [ ]:
len(preds)

In [ ]:
print('test accuracy:', np.mean(answers == preds))

# Make KD dataset

In [12]:
all_files = config['train_data_files'] + config['valid_data_file'] + config['sample_data_file'] + config['test_data_file']

In [13]:
WIDTH = 32
HEIGHT = 32
RGB_MEAN = [125.31, 122.95, 113.87]
RGB_STD = [62.99, 62.09, 66.70]

In [14]:
def _preprocess(image):
  """Perform standard CIFAR preprocessing.

  Pads the image then performs a random crop.
  Then, image is flipped horizontally randomly.

  Args:
    image: image Tensor with shape [height, width, 3]

  Returns:
    preprocessed image with the same dimensions.
  """
  # Pad 4 pixels on all sides with 0
  image = tf.image.resize_image_with_crop_or_pad(
      image, HEIGHT + 8, WIDTH + 8)

  # Random crop
  image = tf.random_crop(image, [HEIGHT, WIDTH, 3], seed=0)

  # Random flip
  image = tf.image.random_flip_left_right(image, seed=0)

  return image


def _parser(use_preprocessing, serialized_example):
  """Parses a single tf.Example into image and label tensors."""
  features = tf.parse_single_example(
      serialized_example,
      features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),
      })
  image = tf.decode_raw(features['image'], tf.uint8)
  image.set_shape([3 * HEIGHT * WIDTH])
  image = tf.reshape(image, [3, HEIGHT, WIDTH])
  # TODO(chrisying): handle NCHW format
  image = tf.transpose(image, [1, 2, 0])
  image = tf.cast(image, tf.float32)
  if use_preprocessing:
    image = _preprocess(image)
  image -= tf.constant(RGB_MEAN, shape=[1, 1, 3])
  image /= tf.constant(RGB_STD, shape=[1, 1, 3])
  label = tf.cast(features['label'], tf.int32)
  return image, label

In [15]:
def _set_batch_dimension(batch_size, images, labels):
    images.set_shape(images.get_shape().merge_with(
      tf.TensorShape([batch_size, None, None, None])))
    labels.set_shape(labels.get_shape().merge_with(
      tf.TensorShape([batch_size])))

    return images, labels


In [16]:
def _dummy_imput_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(params['file'])
    dataset = dataset.prefetch(buffer_size=batch_size)

   
   # Parse, preprocess, and batch images
    parser_fn = functools.partial(_parser, False)
    dataset = dataset.apply(
        tf.contrib.data.map_and_batch(
            parser_fn,
            batch_size=batch_size,
            num_parallel_batches=None,
            drop_remainder=True))

    # Assign static batch size dimension
    dataset = dataset.map(functools.partial(_set_batch_dimension, batch_size))

    # Prefetch to overlap in-feed with training
    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
    

    return dataset

In [17]:
file = config['sample_data_file'][0]
raw_dataset = tf.data.TFRecordDataset([file])
i = 0
for r in raw_dataset:
    i += 1
    
params = {'file':file, 'use_KD':False}
est = tf.contrib.tpu.TPUEstimator(
      use_tpu=False,
      model_fn=model_builder.build_model_fn(spec, config, None),
      config=run_config,
      params=params,
      train_batch_size=config['batch_size'],
      eval_batch_size=config['batch_size'],
      predict_batch_size=100)


est_preds = est.predict(input_fn=_dummy_imput_fn, yield_single_examples=False)

all_pred_logits_aug = []
for preds in est_preds:
    all_pred_logits_aug.append(preds['logits'])
all_pred_logits_aug = np.vstack(all_pred_logits_aug)

Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7f69b5e40950>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Whitelisted: <function _create_dataset_reader.<locals>.read_one_file at 0x7f69b5e40950>: DoNotConvert rule for tensorflow
Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7f69b5e4f1e0>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Whitelisted: <function _create_dataset_reader.<locals>.read_one_file at 0x7f69b5e4f1e0>: DoNotConvert rule for tensorflow
Converted call: functools.partial(<function _parser at 0x7f69b5e40598>, False)
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Not whitelisted: <method-wrapper '__call__' of functools.partial object at 0x7f69b5e50d18>: default rule
Not whitelisted: functools.partial(<function _parser at 0x7f69b5e40598>, False): default rule
Entity <function _parser at 0x7f69b5e40598> is not cached for key <code ob

In [18]:
preds

{'logits': array([[ 2.21688824e+01, -1.08879447e+00, -8.42986166e-01,
         -5.33338356e+00, -6.07698321e-01, -6.71051121e+00,
         -6.49970770e+00, -1.90309179e+00, -1.68296006e-02,
          1.04996431e+00],
        [ 1.64300900e+01, -2.58241034e+00,  2.40355301e+00,
         -3.71633053e+00,  3.80486917e+00, -8.77362061e+00,
         -2.49105883e+00, -2.32892776e+00,  7.81713203e-02,
         -2.48389840e+00],
        [ 7.68267727e+00, -3.35109663e+00, -3.04383063e+00,
         -3.50239968e+00,  1.69831145e+00, -1.49305427e+00,
          2.16184950e+00,  1.26342213e+00, -6.13765287e+00,
          5.25739908e+00],
        [ 1.80912151e+01,  2.24879527e+00,  4.35669720e-01,
         -3.61780548e+00, -2.98626161e+00, -3.06833839e+00,
         -1.66740048e+00, -5.04647493e+00, -2.28480816e+00,
         -1.83047831e+00],
        [ 2.04056797e+01,  4.97365892e-01, -2.58156037e+00,
         -5.11066437e+00, -1.76235688e+00, -4.99113607e+00,
         -5.87872076e+00, -2.65978098e+00,

In [ ]:
print(i, len(all_pred_logits_aug), i == len(all_pred_logits_aug))

In [22]:
hstr = []
for file in all_files:
#for file in ['sample.tfrecords']:
    raw_dataset = tf.data.TFRecordDataset([file])
#     params = {'file':file, 'use_KD':False}
#     est = tf.contrib.tpu.TPUEstimator(
#           use_tpu=False,
#           model_fn=model_builder.build_model_fn(spec, config, None),
#           config=run_config,
#           params=params,
#           train_batch_size=config['batch_size'],
#           eval_batch_size=config['batch_size'],
#           predict_batch_size=100)


#     est_preds = est.predict(input_fn=_dummy_imput_fn, yield_single_examples=False)

#     all_pred_logits_aug = []
#     for preds in est_preds:
#         all_pred_logits_aug.append(preds['logits'])
#     all_pred_logits_aug = np.vstack(all_pred_logits_aug)

#     out_file = file.replace('.', '_KD.')

#     with tf.python_io.TFRecordWriter(out_file) as record_writer:
    i = 0
    for raw_record in raw_dataset:
        
#         example = tf.train.Example()
#         example.ParseFromString(raw_record.numpy())
#         img = example.features.feature['image']
#         label = example.features.feature['label']
#         preds = all_pred_logits_aug[i]
#         new_label = np.hstack((label.int64_list.value[0], preds))
#         feat_preds = tf.train.Feature(float_list=tf.train.FloatList(value=new_label))
#         example = tf.train.Example(features=tf.train.Features(
#                   feature={
#                       'image': img,
#                       'label': feat_preds
#                   }))
        #print(feat_preds)
#         record_writer.write(example.SerializeToString())
        i += 1
#             if i > 2:
#                 break
    hstr.append((file, i))
    

Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8756510>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Whitelisted: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8756510>: DoNotConvert rule for tensorflow
Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8990378>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Whitelisted: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8990378>: DoNotConvert rule for tensorflow
Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8697ea0>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Whitelisted: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b8697ea0>: DoNotConvert rule for tensorflow
Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7f68b89901e0>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=

In [23]:
hstr

[('../data/dataset/train_1.tfrecords', 10000),
 ('../data/dataset/train_2.tfrecords', 10000),
 ('../data/dataset/train_3.tfrecords', 10000),
 ('../data/dataset/train_4.tfrecords', 10000),
 ('../data/dataset/validation.tfrecords', 10000),
 ('../data/dataset/sample.tfrecords', 100),
 ('../data/dataset/test.tfrecords', 10000)]

In [ ]:
logits = tf.convert_to_tensor(np.random.rand(64, 10))

In [ ]:
logits.shape

In [ ]:
labels = tf.convert_to_tensor(np.random.rand(64, 11))

In [ ]:
loss_soft = tf.keras.losses.KLD(
            tf.math.log_softmax(logits),
            tf.math.softmax(labels[:, 1:])
        )

In [ ]:
tf.math.reduce_mean(loss_soft)

In [ ]:
loss_ce = tf.losses.softmax_cross_entropy(
            onehot_labels=tf.one_hot(tf.dtypes.cast(labels[:, 0], tf.int32), config['num_labels']),
            logits=logits)

In [ ]:
loss_ce

In [ ]:
all_pred_logits_aug

In [ ]:
se = example.SerializeToString()

In [ ]:
tse = tf.convert_to_tensor(se)

In [ ]:
tse

In [ ]:
features = tf.parse_single_example(
      tse,
      features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([11], tf.float32),
      })


In [ ]:
features